<a href="https://colab.research.google.com/github/Kodro23/yoda_says/blob/main/Training_text_generation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieve the dialogue

# Train the model

Mainly followed the instructions on the unsloth github README: https://github.com/unslothai/unsloth

In [ ]:
import torch

from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DataCollatorWithPadding,Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset as HfDataset

In [ ]:
#train and test
#train and test
tokenized_dialogues = []
yoda_dialogues = []
for dialogue in dialogues["YODA"]:
    yoda_dialogues.append(dialogue)
    tokens = tokenizer(dialogue, return_tensors='pt', padding="max_length", truncation=True, max_length=128)
    tokens['labels'] = tokens['input_ids'].clone()
    tokenized_dialogues.append({k: v.squeeze(0) for k, v in tokens.items() if k in ['input_ids', 'attention_mask', 'labels']})

#train test split
train_encodings, eval_encodings, train_dialogues, eval_dialogues = train_test_split(tokenized_dialogues, yoda_dialogues, test_size=0.2, random_state=42)
class YodaDataset(HfDataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, idx):
        return {key: torch.tensor(val) for key, val in self.encodings[idx].items()}



train_dataset = YodaDataset(train_encodings)
eval_dataset = YodaDataset(eval_encodings)
# use the DataCollatorWithPadding to pad each batch during training.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [ ]:
# Set up the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory for model checkpoints
    evaluation_strategy="steps",     # evaluate every few steps
    learning_rate=5e-5,              # learning rate for optimizer
    per_device_train_batch_size=2,   # batch size for training
    num_train_epochs=3,              # number of training epochs
    save_steps=10_000,               # save checkpoints every 10,000 steps
    save_total_limit=2,              # only keep the 2 most recent checkpoints
    logging_dir='./logs',            # directory to save logs
    logging_steps=500,               # log every 500 steps
    report_to="none"
)

# Set up the Trainer
trainer = Trainer(
    model=model,                         # the model to train
    args=training_args,                  # training arguments
    train_dataset=train_dataset,          # training data
    eval_dataset=eval_dataset,             # evaluation data
    data_collator=data_collator
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-37-c7a432cd1a19>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val) for key, val in self.encodings[idx].items()}


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


TrainOutput(global_step=96, training_loss=0.7531743844350179, metrics={'train_runtime': 559.7073, 'train_samples_per_second': 0.343, 'train_steps_per_second': 0.172, 'total_flos': 12542017536000.0, 'train_loss': 0.7531743844350179, 'epoch': 3.0})

In [ ]:
# Generate text with the fine-tuned model
input_prompt = "You: Am I or not?\nYoda:"
inputs = tokenizer(input_prompt, return_tensors='pt', padding=True, truncation=True)

# Ensure the attention mask is included
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Generate text
generated_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=50,           # Adjust this to a reasonable number
    num_return_sequences=1,
    temperature=0.7,         # Lower values make the output deterministic; higher values add randomness
    top_p=0.9,               # Nucleus sampling for creativity
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)



# Decode the generated text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(generated_text)

You: Am I or not?
Yoda:


In [ ]:
print("Pad Token ID:", tokenizer.pad_token_id)
print("EOS Token ID:", tokenizer.eos_token_id)


Pad Token ID: 50256
EOS Token ID: 50256
